In [21]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3
import time  # para evitar saturar la API

print("✅ Librerías importadas.")  

✅ Librerías importadas.


In [22]:
pd.set_option('display.max_columns', None) ## muestra todos los datos

In [ ]:
base_url = "https://www.datos.gov.co/resource/rpmr-utcd.json"
limit = 50000  # máximo recomendado por Socrata
offset = 0
all_data = []

print("📥 Extrayendo datos en bloques...")

while True:
    url = f"{base_url}?$limit={limit}&$offset={offset}"
    print(f"🔄 Descargando desde offset {offset}...")
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        chunk = response.json()

        if not chunk:
            break  # cuando no haya más datos, detenemos el ciclo

        all_data.extend(chunk)
        offset += limit
        time.sleep(0.5)  # buena práctica: evitar saturar la API

    except requests.exceptions.RequestException as e:
        print(f"❌ Error en la petición: {e}")
        break

df_raw = pd.DataFrame(all_data)
print(f"✅ ¡Datos cargados! Total de filas: {len(df_raw)}")


📥 Extrayendo datos en bloques...
🔄 Descargando desde offset 0...
🔄 Descargando desde offset 50000...
🔄 Descargando desde offset 100000...
🔄 Descargando desde offset 150000...
🔄 Descargando desde offset 200000...
🔄 Descargando desde offset 250000...
🔄 Descargando desde offset 300000...
🔄 Descargando desde offset 350000...
🔄 Descargando desde offset 400000...
🔄 Descargando desde offset 450000...
🔄 Descargando desde offset 500000...
🔄 Descargando desde offset 550000...
🔄 Descargando desde offset 600000...
🔄 Descargando desde offset 650000...
🔄 Descargando desde offset 700000...
🔄 Descargando desde offset 750000...
🔄 Descargando desde offset 800000...
🔄 Descargando desde offset 850000...
🔄 Descargando desde offset 900000...
🔄 Descargando desde offset 950000...
🔄 Descargando desde offset 1000000...
🔄 Descargando desde offset 1050000...
🔄 Descargando desde offset 1100000...
🔄 Descargando desde offset 1150000...
🔄 Descargando desde offset 1200000...
🔄 Descargando desde offset 1250000...
🔄 Des

In [19]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   nivel_entidad                500000 non-null  object
 1   codigo_entidad_en_secop      500000 non-null  object
 2   nombre_de_la_entidad         500000 non-null  object
 3   nit_de_la_entidad            500000 non-null  object
 4   departamento_entidad         500000 non-null  object
 5   municipio_entidad            500000 non-null  object
 6   estado_del_proceso           500000 non-null  object
 7   modalidad_de_contrataci_n    500000 non-null  object
 8   objeto_a_contratar           499992 non-null  object
 9   objeto_del_proceso           499942 non-null  object
 10  tipo_de_contrato             500000 non-null  object
 11  fecha_de_firma_del_contrato  468580 non-null  object
 12  fecha_inicio_ejecuci_n       468580 non-null  object
 13  fecha_fin_ejec

In [9]:
fechas = [
    'fecha_de_firma_del_contrato',
    'fecha_inicio_ejecuci_n',
    'fecha_fin_ejecuci_n'
]

for col in fechas:
    df_raw[col] = pd.to_datetime(df_raw[col], errors='coerce')


In [11]:
# Elimina caracteres no numéricos y convierte
df_raw['valor_contrato'] = (
    df_raw['valor_contrato']
    .str.replace(r'[^\d.]', '', regex=True)
    .astype(float)
)

In [13]:
df_raw.columns = (
    df_raw.columns
    .str.lower()
    .str.replace('á', 'a')
    .str.replace('é', 'e')
    .str.replace('í', 'i')
    .str.replace('ó', 'o')
    .str.replace('ú', 'u')
    .str.replace('ñ', 'n')
    .str.replace(' ', '_')
)


In [ ]:
df_contrario = df_raw[
    (df_raw['valor_contrato'] <= 0) | 
    (df_raw['fecha_de_firma_del_contrato'].isna())
].copy()

df_contrario.head(1000)

,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,tipo_de_contrato,fecha_de_firma_del_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor
2,Territorial,705228484,COLEGIO REPUBLICA DE MEXICO IED,830052343,Distrito Capital de Bogotá,Bogotá,Cancelado,Selección Abreviada de Menor Cuantía,NO DEFINIDO,Sin Descripcion,Prestación de servicios,NaT,NaT,2024-11-14,CO1.PCCNTR.6420701,CO1.PCCNTR.6420701,48500000.0,LUXE BH TRANSPORTS SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,901536964
56,Territorial,702380031,CUNDINAMARCA-ALCALDIA MUNICIPIO MOSQUERA,899999342,Cundinamarca,Mosquera,Activo,Contratación directa,El Municipio de Mosquera Cundinamarca en calid...,El Municipio de Mosquera Cundinamarca en calid...,Comodato,2023-03-06,NaT,2023-12-13,CO1.PCCNTR.4021451,C.COMODA-003-2022,0.0,SUNET,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,900481871
73,Nacional,703017228,ESTABLECIMIENTO PENITENCIARIO DE MEDIANA SEGUR...,818000409,Chocó,Quibdó,Cancelado,Mínima cuantía,NO DEFINIDO,Sin Descripcion,Suministros,NaT,NaT,2020-08-06,CO1.PCCNTR.1663901,CO1.PCCNTR.1663901,215525.0,PAPELERIA EL CID S.A.S.,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,800021033
78,Nacional,700662059,FONDO ROTATORIO DEL DANE (FONDANE),800072977,Distrito Capital de Bogotá,Bogotá,Borrador,Contratación directa,42795_43287_43288_CONV_MINTIC_19_2020_DIG_TH_P...,42795_43287_43288_CONV_MINTIC_19_2020_DIG_TH_P...,Prestación de servicios,NaT,2020-01-31,2020-02-29,CO1.PCCNTR.1331091,CO1.PCCNTR.1331091,3156000.0,DERLY,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,1032412457
79,Nacional,701174138,CAJA DE RETIRO DE LAS FUERZAS MILITARES,899999118,Distrito Capital de Bogotá,Bogotá,Borrador,Contratación directa,NO DEFINIDO,Sin Descripcion,Prestación de servicios,NaT,NaT,2022-05-30,CO1.PCCNTR.3439011,CO1.PCCNTR.3439011,0.0,NO DEFINIDO,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Sin Descripcion,No Definido


In [ ]:
df = df_raw[
    (df_raw['valor_contrato'] > 0) &
    (df_raw['fecha_de_firma_del_contrato'].notna())
].copy()


In [14]:
df_raw.head()

,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,tipo_de_contrato,fecha_de_firma_del_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor
0,Territorial,702413246,GOBERNACION DEL DEPARTAMENTO DEL CESAR,892399999,Cesar,Valledupar,Cerrado,Contratación directa,PRESTACION DE SERVICIO DE UN ADMINISTRADOR DE ...,PRESTACION DE SERVICIO DE UN ADMINISTRADOR DE ...,Prestación de servicios,2023-06-07,2023-06-08,2023-12-22,CO1.PCCNTR.5052241,2023 02 1362,27343024.0,LLERALDIN VIVEROS RAMOS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,1065612378
1,Territorial,718600463,UNIATLANTICO,890102257,Atlántico,Puerto Colombia,En ejecución,Contratación régimen especial,PRESTACIÓN DE LOS SERVICIOS DE SALUD A USUARIO...,PRESTACIÓN DE LOS SERVICIOS DE SALUD A USUARIO...,Prestación de servicios,2024-10-28,2024-11-06,2024-12-31,CO1.PCCNTR.6874537,C-US-065-2024,300000000.0,OINSAMED SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,9004653194
2,Territorial,705228484,COLEGIO REPUBLICA DE MEXICO IED,830052343,Distrito Capital de Bogotá,Bogotá,Cancelado,Selección Abreviada de Menor Cuantía,NO DEFINIDO,Sin Descripcion,Prestación de servicios,NaT,NaT,2024-11-14,CO1.PCCNTR.6420701,CO1.PCCNTR.6420701,48500000.0,LUXE BH TRANSPORTS SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,901536964
3,TERRITORIAL,244801011,LA GUAJIRA - ALCALDÍA MUNICIPIO DE ALBANIA,NO DEFINIDO,La Guajira,Albania,Celebrado,Contratación Directa (Ley 1150 de 2007),PRESTACION DE SERVICIOS DE FORTALECIMIENTO PAR...,PRESTACIoN DE SERVICIO DE FORTALECIMIENTO PARA...,Prestación de Servicios,2023-09-21,2023-10-01,2024-01-01,23-12-13802690,CDIR 131-23,240000000.0,CUERPO DE BOMBEROS VOLUNTARIOS DE ALBANIA  LA...,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Cédula de Ciudadanía,8785701
4,Territorial,702730482,SUBRED INTEGRADA DE SERVICIOS DE SALUD SUR E.S.E.,900958564,Distrito Capital de Bogotá,Bogotá,Modificado,Contratación régimen especial,PRESTAR SERVICIOS COMO PROFESIONAL PARA EL ENT...,PRESTAR SERVICIOS COMO PROFESIONAL PARA EL ENT...,Prestación de servicios,2020-11-04,2020-11-04,2021-04-30,CO1.PCCNTR.1960619,6802-2020,21597378.0,farid steven rodriguez vega,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,1012422618
